In [1]:
from mlflow.tracking import MlflowClient
MLFLOW_SET_TRACKING_URI = "http://127.0.0.1:5000"
client = MlflowClient(tracking_uri=MLFLOW_SET_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1', creation_time=1746683774913, experiment_id='1', last_update_time=1746683774913, lifecycle_stage='active', name='ny-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/0', creation_time=1746683599358, experiment_id='0', last_update_time=1746683599358, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
client.create_experiment(name="my-cool-experiment")

'2'

In [10]:
#List runs on the basis of top 10 lowes rmse with experiment id 1 that is ny_taxi
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
     max_results=5,
    order_by=["metrics.rmse ASC"] #
)

In [11]:
runs

[<Run: data=<RunData: metrics={'rmse': 5.809681017912009}, params={'learning_rate': '0.09064205157889488',
  'max_depth': '22',
  'min_child_weight': '1.0629059550227644',
  'objective': 'reg:linear',
  'reg_alpha': '0.13507272239403997',
  'reg_lambda': '0.01337017283417198',
  'seed': '42'}, tags={'developer': 'Shakti',
  'mlflow.runName': 'blushing-gnat-647',
  'mlflow.source.name': '/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts', end_time=1746811838007, experiment_id='1', lifecycle_stage='active', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_name='blushing-gnat-647', run_uuid='bad7bfb0e73340a6bd111039cdfd4375', start_time=1746811700264, status='FINISHED', user_id='codespace'>, inputs=<RunInputs: dataset

In [12]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: bad7bfb0e73340a6bd111039cdfd4375, rmse: 5.8097
run id: 022ccf19e9f244548dfe3e4571551726, rmse: 5.8119
run id: 3d51f711bda342be9ef0b6e36113fddc, rmse: 5.8141
run id: 811b89f631de4672a48ceb8ea0526e40, rmse: 5.8236
run id: d3a43abd35a04b18b51f096b004e348b, rmse: 5.8237


In [4]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_SET_TRACKING_URI)

In [5]:
#Found the best runid and registered the model ran on that runid
run_id = "bad7bfb0e73340a6bd111039cdfd4375"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/05/13 08:34:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1747125264621, current_stage='None', description='', last_updated_timestamp=1747125264621, name='nyc-taxi-regressor', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_link='', source='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts/model', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [8]:
model_name = "nyc-taxi-regressor"


# Fetch all versions of the model
versions = client.search_model_versions(f"name='{model_name}'")

for v in versions:
    print(f"Version: {v.version}, Status: {v.status}, Stage: {v.current_stage}")


Version: 2, Status: READY, Stage: None
Version: 1, Status: READY, Stage: None


In [12]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
model_name = "nyc-taxi-regressor"
model_version = "2"

# Set a tag to indicate stage (e.g., "promoted_to" or "status")
client.set_model_version_tag(
    name=model_name,
    version=model_version,
    key="promoted_to",
    value="staging"
)

# Optionally, update description
client.update_model_version(
    name=model_name,
    version=model_version,
    description="Promoted to staging after evaluation"
)


<ModelVersion: aliases=[], creation_timestamp=1747125264621, current_stage='None', description='Promoted to staging after evaluation', last_updated_timestamp=1747127687704, name='nyc-taxi-regressor', run_id='bad7bfb0e73340a6bd111039cdfd4375', run_link='', source='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/1/bad7bfb0e73340a6bd111039cdfd4375/artifacts/model', status='READY', status_message=None, tags={'promoted_to': 'staging'}, user_id='', version='2'>

In [13]:
#Testing if prod model is woking fine with latest data if yes it will be there else it will be replaced

from sklearn.metrics import mean_squared_error
import pandas as pd

def read_dataframe(filename):
    df = pd.read_parquet(filename,engine='pyarrow')

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transfrom(train_dicts)


def test_model(stage, X_test, y_test, name='nyc-taxi-regressor'):
    model = mlflow.pfunc.load_model(f"models/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test,y_pred, squared=False)}


In [14]:
df= read_dataframe('./data/green_tripdata_2021-02.parquet')



In [ ]:
client.download_artifacts(run_id=run_id)